# Pandas assignment

The problems in this notebook are adapted from Brandon Rhodes's Pycon `pandas` tutorial.

The first few problems are identical.

## Loading the data

This notebook loads two datasets `titles` and ` cast`.  Both are are loaded frim the course website and contain extracts of the IMDB movie data. The second is a fragment of a much larger DataFrame.  

Everything
you need to complete this assignment is included in the **fragment loaded in this section**
in the cells below.

For those who want to have the entire IMDB-derived dataset, here are some pointers.

Loading the entire IMDB data set used for these tutorial exercises is best done by
visiting [Brandon Rhodes' github repo](https://github.com/brandon-rhodes/pycon-pandas-tutorial)
and following the instructions.  You can either download a big zip file or, if you have `git` (the main
github API function) installed on your machine, do a `git clone` command.
In addition to that,
you will need to download 4 compressed files by ftp that Rhodes
provides links to.  You can then run code
from Rhodes' cloned repo to create the non-truncated version CSV files used in these exercises.
If you do that, you may use the complete dataset to complete these exercises.  Some but not
all of your answers will be dufferent.

The statistics
in the Part B answers will not be correct until you load the complete dataset.  But
you can complete the assignment on the fragment, since I will be evaluating your code,
not the accuracy of your statistics.

#### Context Cell (Parameter-setting and Notebook-wide imports)

In [3]:
# The Python modules you need for this assignment.
import pandas as pd
import os.path
import urllib.request
import urllib.error
from matplotlib import pyplot as plt

# Switch to True if loading the data below yields a 404 Not Found Error
data_load_has_failed = False

github_url ='https://raw.githubusercontent.com/gawron/python-for-social-science/master/'
#truncated_data_path = 'pandas/tutorial/pycon-pandas-tutorial-master/truncated_data/'
truncated_data_path = 'pandas/tutorial/pycon-pandas-tutorial-master/data/'
titles_file = 'titles.csv'
cast_file = 'truncated_cast.csv'

if data_load_has_failed:
    url_dir = github_url + truncated_data_path
else:
    url_dir = 'https://gawron.sdsu.edu/python_for_ss/course_core/data/'

In [8]:
url_dir

'https://gawron.sdsu.edu/python_for_ss/course_core/data/'

Loading and applying the style sheets in the next two cells customizes the styles of your notebook output,
in particular for how pandas `DataFrame`s are printed.

This is optional,  but it's interesting if you know anything about css files.

In [9]:
from IPython.core.display import HTML

if not data_load_has_failed:
    # There are issues with doing it this way on Windows
    #target_url1 = os.path.join(url_dir,'style-notebook.css')
    #target_url2 = os.path.join(url_dir,'style-table.css')
    target_url1 = url_dir + '/style-notebook.css'
    target_url2 = url_dir + '/style-table.css'

    with urllib.request.urlopen(target_url1) as fh1:
        css1 = fh1.read().decode('utf8')
    with urllib.request.urlopen(target_url2) as fh2:
        css2 = fh2.read().decode('utf8')
    css = css1 + css2
    HTML('<style>{}</style>'.format(css))

The next cell loads the `titles` DataFrame, the first of two used in this assignment.  There are only
two columns, `'title'`  and `'year'`.

In [10]:
try:
    titles = pd.read_csv(url_dir + titles_file)
except urllib.error.HTTPError:
    # This uses the same file URL as data_load_has_failed = False
    # To avoid this error trapping in the future, set data_load_has_failed = True
    # in the Context Cell above.
    print("HTTPError Handled!")
    url_dir = github_url + truncated_data_path
    titles = pd.read_csv(url_dir + titles_file)

titles.head()

,title,year
0,The Patriarchs,2009
1,Angels in the Attic,1998
2,The Rapture,1991
3,Star na si Van Damme Stallone,2016
4,Sweet Talk,2004


This is a simple `DataFrame` with two columns, containing the title and year of a film.

If a film is remade and given the same title, the title shows up twice:

In [11]:
titles[titles['title']  == 'Around the World in 80 Days']

,title,year
91875,Around the World in 80 Days,2004
121800,Around the World in 80 Days,1956


The next cell loads the `cast` DataFrame, the second of two used in this assignment.  This is
large and will take a while.

In [12]:
#cast = pd.read_csv('data/cast.csv')
#cast = pd.read_csv(os.path.join(url_dir,'cast.csv'),index_col=0)
cast = pd.read_csv(url_dir + cast_file)
cast.head()

,Unnamed: 0,title,year,name,type,character,n
0,0,In the Land of the Head Hunters,1914,Paddy 'Malid,actor,Kenada,5.0
1,1,The Colour of Darkness,2016,Ketan Daraji -Gohel,actor,Chhagan,NaN
2,2,London Betty,2009,Isaiah Entsua -Mensah,actor,Camera Man,NaN
3,3,Candelabra,2014,Groovin .,actor,Lt. Dick Sims,5.0
4,4,Bad Ideas,2012,Hamid .,actor,The Diner,NaN


In [13]:
len(cast)

422960

In [14]:
cast["title"]

,title
0,In the Land of the Head Hunters
1,The Colour of Darkness
2,London Betty
3,Candelabra
4,Bad Ideas
...,...
422955,Two Way Stretch
422956,Up the Creek
422957,Waltz of the Toreadors
422958,What's New Pussycat


Please have a look at the columns of the `cast` `DataFrame`and make sure you understand what information it contains. A row uniquely identifies a role in a film; it contains the film title, the year the file was made, the name of the actor or actress playing the role, and the name of the character they played.  The number in the `'n'` column represents the importance of the part, with the lead role receiving a 1 and all less important roles receiving higher numbers.

Sometimes extras are included; with extras included, a big Hollywood production can easily  cast over 1000 actors and will therefore populate over 1000 rows of `cast`.  The character name for the extras will be "Extra".

In [ ]:
len(cast[(cast['title']  == 'Around the World in 80 Days')&(cast['year']  == 1956)])

1299

In [ ]:
cast[(cast['title']  == 'Around the World in 80 Days')&(cast['year']  == 1956)].iloc[:5]

,title,year,name,type,character,n
784,Around the World in 80 Days,1956,Ronald Adam,actor,Club Steward,47.0
7147,Around the World in 80 Days,1956,Ray Arnett,actor,Extra,NaN
25665,Around the World in 80 Days,1956,Charles Boyer,actor,Monsieur Gasse - Thomas Cook Paris Clerk,5.0
40928,Around the World in 80 Days,1956,Suey Chan,actor,Extra,NaN
44164,Around the World in 80 Days,1956,Martin Cichy,actor,Bartender,NaN


As shown in the next cell, there are a number of null or `Nan` values in the `'n'` column,
roles for which no ``n`-value is available.

In [ ]:
len(cast[cast['n'].isnull()])

154661

There are also some very
high numbers in the `'n'` column, but not very many of them.

In [ ]:
len(cast[cast['n']>500])

232

FYI:  A single actor may play more than one role in a film, and both those roles can be the starring role.

In [ ]:
cast[(cast['title']=='Around the World in 80 Days')&(cast['year']==2004)&(cast['n']==1)]

,title,year,name,type,character,n
351948,Around the World in 80 Days,2004,Jackie Chan,actor,Passepartout,1.0
351949,Around the World in 80 Days,2004,Jackie Chan,actor,Lau Xing,1.0


## Part A Questions involving selecting and sorting subsets of the rows in the Dataset

### How many movies are listed in the titles dataframe?

As an example, we have put the answer in the next cell.  Be sure to execute the cell containing your solution, so that the answer to the question is displayed in the output as it is in this example.

For each question, first give some thought to which of the two DataFrames loaded in the section entitled "Loading the Data" is best suited to provide an answer.

In [15]:
len(titles)

226013

In [16]:
### What are the earliest two films listed in the titles dataframe?

#Hint:  This will require using `.sort_values(...)`.

In [17]:
earliest = titles.sort_values(by='year').head(2)
earliest

,title,year
82467,Miss Jerry,1894
31347,The Startled Lover,1898


### How many movies have the title "Hamlet"?

Hint: one approach is to use a Boolean mask

In [18]:
hamlet=len(titles[titles['title']=='Hamlet'])
hamlet

20

### How many movies are titled "North by Northwest"?

In [19]:
northwest=len(titles[titles['title']=='North by Northwest'])
northwest

1

### When was the first movie titled "Hamlet" made?

In [20]:
year_hamlet=titles[titles['title']=='Hamlet'].sort_values(by='year').head(1)
year_hamlet

,title,year
9506,Hamlet,1910


### List all of the "Treasure Island" movies from earliest to most recent.

In [21]:
treasure=titles[titles['title']=='Treasure Island'].sort_values(by='year')
treasure

,title,year
96793,Treasure Island,1918
122654,Treasure Island,1920
127163,Treasure Island,1934
169035,Treasure Island,1950
99795,Treasure Island,1972
191251,Treasure Island,1973
135165,Treasure Island,1985
146471,Treasure Island,1999


### How many movies were made in the year 1950?

In [23]:
fifth=len(titles[titles['year']==1950])
fifth

1109

### How many movies were made from 1950 through 1959?

In [24]:
allfifthsmovies = len(titles[(titles['year']>=1950)&(titles['year']<=1959)])
allfifthsmovies

12664

### In what years has a movie titled "Batman" been released?

In [30]:
batman = titles[titles['title']=='Batman']
batman

,title,year
59853,Batman,1943
79855,Batman,1989


### How many credited roles are there in the movie "Inception"?  We'll interpret  this to mean we should not count roles that have no `n`-value.

The idea is that some of the rows have `NaN` in the `'n'` column.  These are not
values.  To eliminate such rows use the `.notnull()` method.

In [31]:
inceptionroles= len(cast[(cast['title']=='Inception')&(cast['n'].notnull())])
inceptionroles

51

### Display the cast of "North by Northwest" in their correct "n"-value order, ignoring roles that did not earn a numeric "n" value.

In [32]:
northwestcast=cast[(cast['title']=='North by Northwest')&(cast['n'].notnull())].sort_values(by='n')
northwestcast

,Unnamed: 0,title,year,name,type,character,n
93664,93664,North by Northwest,1959,Cary Grant,actor,Roger O. Thornhill,1.0
356895,356895,North by Northwest,1959,Eva Marie Saint,actress,Eve Kendall,2.0
157014,157014,North by Northwest,1959,James Mason,actor,Phillip Vandamm,3.0
325773,325773,North by Northwest,1959,Jessie Royce Landis,actress,Clara Thornhill,4.0
38422,38422,North by Northwest,1959,Leo G. Carroll,actor,The Professor,5.0
317316,317316,North by Northwest,1959,Josephine Hutchinson,actress,Mrs. Townsend,6.0
184818,184818,North by Northwest,1959,Philip Ober,actor,Lester Townsend,7.0
137716,137716,North by Northwest,1959,Martin Landau,actor,Leonard,8.0
259201,259201,North by Northwest,1959,Adam Williams,actor,Valerian,9.0
196223,196223,North by Northwest,1959,Edward Platt,actor,Victor Larrabee,10.0


### Now display the entire cast, in "n"-order, of the 2007 version of "Sleuth".

Issue:  There is more than one film with this title.  Use the one made in the year 2007.

In [33]:
sleuth2007=cast[(cast['title']=='Sleuth')&(cast['year']==2007)].sort_values(by='n')
sleuth2007

,Unnamed: 0,title,year,name,type,character,n
35082,35082,Sleuth,2007,Michael Caine,actor,Andrew,1.0
139365,139365,Sleuth,2007,Jude Law,actor,Milo,2.0
195753,195753,Sleuth,2007,Harold Pinter,actor,Man on T.V.,3.0
26741,26741,Sleuth,2007,Kenneth Branagh,actor,Other Man on T.V.,NaN
40180,40180,Sleuth,2007,Alec (II) Cawthorne,actor,Inspector Doppler,NaN
284314,284314,Sleuth,2007,Eve (II) Channing,actress,Marguerite Wyke,NaN
344594,344594,Sleuth,2007,Carmel O'Sullivan,actress,Maggie,NaN


### How many roles were credited in Branagh’s 1996 Hamlet?

Good thing to know:  There was only one film entitled Hamlet made in 1996.

In [34]:
hamlet1996=len(cast[(cast['title']=='Hamlet')&(cast['year']==1996)])
hamlet1996

54

### How many "Hamlet" roles have been listed in all film credits through history?

In [35]:
hamlethistory=len(cast[cast['title']=='Hamlet'])
hamlethistory

342

### How many people have played "James Bond" as a leading role?

Yes, I'm thinking of the Ian Fleming character that has given rise to a whole film franchise,
but I'm going to allow for a little noise.  It
turns out that characters named "James Bond" have come up many times in film history.  Adding
the qualification "as a leading role" helps with that.
This will still leave in a couple of non-Ian Fleming Bonds, but don't worry.
If you just answer the question literally, your
answer should be very close to a list of the actors who've played the
Ian Fleming character.  By the way,  David Niven will be missing from
that list, because the name of the character
listed in the credits of his 1967 parody is not "James Bond" but "Sir James Bond".
It's okay to leave out David Niven.  It was a parody, not a real Bond film.

Note: Relative to this DB (which is dated), the historically correct answer to the question how many actors played the Ian Fleming character is 6. The literal answer to the question in this DB is
greater than 6 because of a few superfluous Bonds, but if your answer is a lot more than 6
(greater than 10), then there's an issue with your code.  There is a technical challenge in this question;
it is tricky to get exactly the right list of actors.  Pay attention to the English.  It's how many peoples, not how many roles.

In [36]:
jamesbondleadingrole = len(cast[(cast['character']=='James Bond')&(cast['n']==1)])
jamesbondleadingrole

26

### How many people have played a role called "The Dude"?

In [37]:
peopleplayedthedude=len(cast[cast['character']=='The Dude'])
peopleplayedthedude

17

### How many roles has Sidney Poitier played throughout his career?

In [38]:
sidneypoitier=len(cast[cast['name']=='Sidney Poitier'])
sidneypoitier

43

### List the supporting roles (having n=2) played by Cary Grant in the 1940s, in order by year.

Yes, Cary Grant did play some supporting roles.  So the answer is not the empty set.

In [39]:
carygrant = cast[(cast['name']=='Cary Grant')&(cast['n']==2)&(cast['year']>=1940)&(cast['year']<1950)].sort_values(by='year')
carygrant

,Unnamed: 0,title,year,name,type,character,n
93661,93661,My Favorite Wife,1940,Cary Grant,actor,Nick Arden,2.0
93671,93671,Penny Serenade,1941,Cary Grant,actor,Roger Adams,2.0


### How many roles were available for actresses in the 1920s?

In [40]:
availableactresses=len(cast[(cast['type']=='actress')&(cast['year']>=1920)&(cast['year']<1930)])
availableactresses

23826

## Part B:  Questions needing value_counts, pivot_tables, or cross tabulation

Brief discussion of an important issue for the data below.

Some of the numbers in the following table are going
to be misleading:

In [ ]:
pt000 = cast.pivot_table(index='title',aggfunc='size').sort_values(ascending=False)

It looks as if this might give us film cast sizes (`.size()` tells how many rows in each group, and rows are roles).

In [ ]:
pt000[:5]

title
Around the World in 80 Days     1364
Hamlet                           342
Mr. Smith Goes to Washington     217
Wells Fargo                      193
Union Pacific                    183
dtype: int64

But that's not quite right: And here's why:

In [ ]:
len(cast[cast['title']=='Around the World in 80 Days'])

1364

In [ ]:
cast[cast['title']=='Around the World in 80 Days']['year']

784       1956
7147      1956
25665     1956
40851     2004
40852     2004
          ... 
422807    1956
422808    1956
422809    2004
422810    1956
422811    1956
Name: year, Length: 1364, dtype: int64

So 1364 is actually the size of the combined cast of two films, one made in 1956 and the other in 2004.  Not whjat we thought we were getting.  Let's call this the **remake** issue.  Bear it in mind as you answer the questions below.

### Of the films made in either 1939 or 1966, what films had a cast size of 90 or more but fewer than 100?


After you've made a DataFrame containing the set of rows in `cast` that you're interested in,
you again need to do a computation that's counts cast members on films.   There is a simple computation  that seems to give you the cast sizes of all films.

But to answer this question correctly,  you will need to
come up with a slightly more complicated answer. The issue to
think about is remakes.  In fact in 1966 there were some remakes of
great 1939 films.  How should that affect your answer to this
problem?


### How many films made before 1939 have a cast of size 1.

### How many movies have had remakes?

Film buffs will know this is much harder than it might seem.  To make this
doable, let's look only at remakes that have the same title, and let's assume (falsely)
that when two movies have the same title, the later one is a remake of the earlier one.

### Plot the number of male and female roles year by year (up through 2017)

So you want years on the x-axis and two lines, one
tracking the number of male roles and another tracking the number of female roles.

Note this plot will not be realistic because of the way our data has been sampled,
so I'll just be evaluating the code, not the accuracy.

### Plot the percentages of all roles that are female year by year for the century from 1917 through 2017

You are continuing your study of the imbalance of male and female roles.  Years on the x-axis,
one line tracking the percentage of female roles.

Comment: This is  a very interesting plot, which is begging for a story to explain it.
On the truncated sample, the plot has some serious flaws, but the general pattern you
see is correct.